<a id="top"></a>
# Question and Answer Model Deployment
*** 
This notebook contains step by step procedue on how to deploy the QnA (BERT) model. It is provided as a function, which lets you replace with your own model if desired as well as ability to customize  

**Note : BERT models are memory intensive, try with bigger environment for smoother performance (8vCPU and 16GB RAM works perfectly)**     

### Create Client Connections to Watson Machine Learning & Set Default Space

In this step we create the Watson Machine Learning client, get the deployment_space_uid, and set the default space.


In [1]:
# WML IMPORT: ACCESS DEPLOYED MODELS
import ibm_watson_machine_learning
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator

# READING AND WRITING PROJECT ASSETS
import project_lib
import sys
project = project_lib.Project()

if '/project_data/data_asset/' not in sys.path:
    sys.path.insert(0, '/project_data/data_asset/')


### Provide the name for space, model and deployment
- Space name is the name given to a space, which is the containerized part of all assets that you push to the deployment section (Watson Machine Learning)
- Model name is the name you want to give to the model
- Deployment name is the name of the deployment of your model

In [2]:
# Specify a name for the space being created, the saved models and the model deployments
space_name = 'WML MSK Benefits'
model_name = 'wml_msk_benefits_model'
deployment_name = 'wml_msk_benefits_model_deployment'

use_existing_space=True

### Connect to the Watson Machine Learning client

In [3]:
from ibm_watson_machine_learning import APIClient
import os

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "3.5"
}

wml_client = APIClient(wml_credentials)

### Create a deployment space
If one with the same name already exists, will use that

In [4]:
space_uid=""
for space in wml_client.spaces.get_details()['resources']:

    if space['entity']['name'] ==space_name:
        print("Deployment space with ",space_name,"already exists . .")
        space_uid=space['metadata']['id']
        wml_client.set.default_space(space_uid)
        if(use_existing_space==False):

            for deployment in client.deployments.get_details()['resources']:
                print("Deleting deployment",deployment['entity']['name'], "in the space",)
                deployment_id=deployment['metadata']['id']
                wml_client.deployments.delete(deployment_id)
            print("Deleting Space ",space_name,)
            wml_client.spaces.delete(space_uid)
            time.sleep(10)
        else:
            print("Using the existing space")
            
            
if (space_uid=="" or use_existing_space==False):
    print("\nCreating a new deployment space -",space_name)
    # create the space and set it as default
    space_meta_data = {
        wml_client.spaces.ConfigurationMetaNames.NAME : space_name

        }

    stored_space_details = wml_client.spaces.store(space_meta_data)

    space_uid = stored_space_details['metadata']['id']

    wml_client.set.default_space(space_uid)


Creating a new deployment space - WML MSK Benefits
Space has been created. However some background setup activities might still be on-going. Check for 'status' field in the response. It has to show 'active' before space can be used. If its not 'active', you can monitor the state with a call to spaces.get_details(space_id)


## Define parameters for the deployment function
These parameters will be utilized in the function below, even when deployed

In [5]:
header = {'Content-Type': 'application/json'}
ai_parms = {'wml_credentials' : wml_credentials,'header' : header}

In [6]:
def msk_wd_pipeline(parms = ai_parms):     
    import subprocess
    import sys

    proc = subprocess.run(['pip','install','pyjwt==2.0.0', 'tensorflow=2.5', 'ibm_watson', 'lime', 'transformers==4.7.0', 'cryptography==3.3.1','--user'],stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    sys.path.insert(0,'/opt/conda3/lib/python3.7/site-packages')
    sys.path.insert(0,'/home/wmlfuser/.local/lib/python3.7/site-packages/')
    modules = [key for key in sys.modules.keys() if 'jwt' in key ]
    for module in modules:
        del sys.modules[module]
    import jwt
    import importlib
    importlib.reload(jwt)
    import pkg_resources
    importlib.reload(pkg_resources)
    from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
    import sys    
    import os
    import json
    import sys
    import pandas as pd
    import numpy as np
    import time
    import random
    import requests
    import json
    import subprocess
    from collections import OrderedDict
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    
     # GENERAL PYTHON IMPORTS

    import uuid
    from tqdm import tqdm_notebook as tqdm
    import datetime
    import random
    import matplotlib.pyplot as plt
    
    from ibm_cloud_sdk_core.authenticators import BearerTokenAuthenticator
    from ibm_watson import DiscoveryV2
    
    #NLU Packages
    import json
    from ibm_watson import NaturalLanguageUnderstandingV1
    from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
    from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions



    # WML IMPORT: ACCESS DEPLOYED MODELS
    import ibm_watson_machine_learning
    
    import base64
    from requests.auth import HTTPBasicAuth
    import warnings
    warnings.filterwarnings("ignore", message="numpy.dtype size changed")
    warnings.filterwarnings("ignore")
    
    #BERT
    import pandas as pd
    import numpy as np
    from transformers import pipeline
    
    #NOTE : The model defined here can be changed to your desired model if needed
    bert_lg_cased = pipeline('question-answering', model = 'bert-large-cased-whole-word-masking-finetuned-squad')

    #WARNING : Make sure you change the below 2 variables depending on your environment and project ID
    url = "https://zen.msk.datascienceelite.com/discovery/zen-wd/instances/1626894389011290/api"
    projID = "3195d293-5c34-46dc-b573-a9f01fd60bce"
    
   

    def generate_access_token():
        headers={}
        headers["Accept"] = "application/json"
        #WARNING : Replace with your username and password
        auth = HTTPBasicAuth("rakshith.dasenahalli.lingaraju@ibm.com", "IBMDSE2021!")
        
        #WARNING : Replace with your environment url (https://<YOUR URL>/v1/preauth/validateAuth)
        ICP_TOKEN_URL= "https://zen.msk.datascienceelite.com/v1/preauth/validateAuth"
    
        response = requests.get(ICP_TOKEN_URL, headers=headers, auth=auth, verify=False)
        json_data = response.json()
        icp_access_token = json_data['accessToken']
        return icp_access_token
    
    def bert_func(question_, context_):
        import re
        from bs4 import BeautifulSoup
        answer = bert_lg_cased(question=question_, context=context_, max_answer_len = 300,
                       max_seq_len = 512) #topk=5
        indexes = [x.start() for x in re.finditer('\.', context_)]
        if [i for i in indexes if i<=answer['start']]:
            start_ = max([i for i in indexes if i<=answer['start']]) + 2
        else:
            start_ = 0
        end_ = min([i for i in indexes if i>=answer['end']])
        answer['expanded_answer'] = context_[start_ : end_]
    
        return answer   

    def score( payload ):
        import json
        global client
        import requests
        import time
        import re
        from bs4 import BeautifulSoup
        t1 = time.time()
        token_ = generate_access_token()
        authenticator = BearerTokenAuthenticator(token_)
        discovery = DiscoveryV2(version='2019-11-22', authenticator=authenticator)
        discovery.set_service_url(url)
        discovery.set_disable_ssl_verification(True)
        response_list = discovery.list_collections(project_id = projID).get_result()
        collectionID = response_list["collections"][0]["collection_id"]
        t2 = time.time()
        
        #NLU 
        question=payload["input_data"][0]["values"]
        #WARNING : Replace with your NLU auhtentication token
        nlu_authenticator = IAMAuthenticator('sqszQRzXz9OwR2-hT0UDq5GGWbrd1JMWX0IYqFSB97sC') #API key from the NLU service on SaaS
        natural_language_understanding = NaturalLanguageUnderstandingV1(
            version='2021-03-25',
            authenticator=nlu_authenticator)
        #WARNING : Replace with your NLU URL
        natural_language_understanding.set_service_url('https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/b89a1195-ba83-44d6-bc37-dd9b8cfeaceb')
        natural_language_understanding.set_disable_ssl_verification(True)
        response = natural_language_understanding.analyze(
        text=question,
        #WARNING : Replace with your WKS model ID being used by the NLU for finding entities from user questions
        features=Features(
            entities=EntitiesOptions(model="b63a2f79-6ed5-4490-8dc9-e57076c5aa1b", emotion=False, sentiment=False, limit=10), # modelID of the model created and deployed from WKS
            )

        ).get_result()
        intents = [i['type'] + ' - ' + i['text'] for i in response['entities']]
#        print("init time : " + str(t2-t1))
#        print(collectionID)
        t3 = time.time()
        print(t3-t2)

        
#        print(question)
        #WARNING : Replace with your WD project ID
        response = discovery.query(
        project_id=projID,collection_ids=["87d0255d-87d2-596b-0000-017b0e0f465e"],
        natural_language_query= question,
        passages= {"enabled": True, "find_answers":True, "per_document":True}).get_result()
        
        _list = []
        _url = []
        for r in response['results']:
            for s in r['document_passages']: 
                if s['field'] == 'text':
                    cleantext = BeautifulSoup(s['passage_text'], "lxml").text
                    new_p = re.sub("\s+"," ", cleantext)        
                    new_p = new_p + '.' if (not new_p.endswith('.')) else new_p
                    new_p = new_p[0].upper() + new_p[1:]
                    u_ = r['metadata']['source']['url']
                    if new_p not in _list:
                        _list.append(new_p)
                        _url.append(r['metadata']['source']['url'])

        context = ''
        #Taking only top 5 passages from WD to be passed to BERT
        _list = _list[0:5]
        _url = _url[0:5]
        t4 = time.time()
#        print("query time : " + str(t4-t3))
        t5= time.time()
        bert_answers = []
        for i,j in zip(_list,_url):
#            print(i,j)
            context = re.sub(r"\s+", " ", i)
            ba = bert_func(question, context)
            ba['context'] = context
            ba['url'] = j
            bert_answers.append(ba)
        sorted_answers = sorted(bert_answers, key = lambda i: i['score'], reverse=True)
        # Picking only top 3 answers from BERT for display
        sorted_answers = sorted_answers[0:3]

        result = {}
        result["fields"] = ["answers"]
        result["values"] = [sorted_answers]
        result['values'] = [intents] + result['values']
       
        formatted_result= {}
        formatted_result['predictions']=[result]
        t6 = time.time()
#        print("answer time : " + str(t6-t5))
        return formatted_result

    return score




### Run a test to see if the function is working correctly

In [7]:
from IPython.display import clear_output
!pip install ibm_watson
!pip install transformers


request_data = {"values": 'What expenses are covered by vision plan?', "fields": "question"}

payload = {"input_data": [request_data]}


answer = msk_wd_pipeline()(payload)
clear_output()

In [8]:
answer

{'predictions': [{'fields': ['answers'],
   'values': [['health - vision', 'health - covered'],
    [{'score': 0.4368143081665039,
      'start': 89,
      'end': 95,
      'answer': 'vision',
      'expanded_answer': 'You can use to pay eligible out-of-pocket healthcare (medical, prescription, dental, and vision) expenses now or in the future',
      'context': 'You can use to pay eligible out-of-pocket healthcare (medical, prescription, dental, and vision) expenses now or in the future. You’re only eligible for an HSA if you’re enrolled in the UHC CDHP. If you enroll in a different medical plan, or waive medical coverage, the Health Care Flexible Spending Account (FSA) is a great alternative.',
      'url': 'https://mskbenefits.mskcc.org/new-hires/tour'},
     {'score': 0.35381755232810974,
      'start': 34,
      'end': 62,
      'answer': 'eligible deductible expenses',
      'expanded_answer': 'I am requesting reimbursement for eligible deductible expenses incurred by insurance c

### Define the Envrionment YAML definitions

In [9]:
%%writefile environment.yml
channels:
  - empty
  - nodefaults

dependencies:
  - pip:
    - ibm_watson
    - pandas
    - numpy 
    - protobuf==3.11.2
    - tensorflow==2.4.1
    - transformers==4.7.0
    - lxml

Writing environment.yml


### Define metadata for the package extension

In [10]:
meta_props = {
   wml_client.package_extensions.ConfigurationMetaNames.NAME: "WDMSK",
   wml_client.package_extensions.ConfigurationMetaNames.TYPE: "conda_yml",
}

pkg_extn_details = wml_client.package_extensions.store(meta_props, "./environment.yml")
pkg_extn_id = wml_client.package_extensions.get_id(pkg_extn_details)

Creating package extensions
SUCCESS


### Define name for your function and the deployment of that function

In [11]:
FUNCTION_NAME = 'MSK_WD_Wrapper_Bert'
FUNCTION_DEPLOYMENT_NAME = 'msk_wd_pipeline_deployment_bert'

### Check and delete if any previous deployment with the same name exists

In [12]:
deployment_details = wml_client.deployments.get_details()
for deployment in deployment_details['resources']:
    f_deployment_id = deployment['metadata']['id']
    f_id = deployment['entity']['asset']['id']
    if deployment['entity']['name'] == FUNCTION_DEPLOYMENT_NAME:
        print('Deleting function deployment id', f_deployment_id)
        wml_client.deployments.delete(f_deployment_id)
        print('Deleting function id', f_id)
        wml_client.repository.delete(f_id)

### Store the function to the defined space

In [14]:
base_id = wml_client.software_specifications.get_uid_by_name('default_py3.7')
print("base_id", base_id)
meta_props = {
   wml_client.software_specifications.ConfigurationMetaNames.NAME: "mskrdlv5",
   wml_client.software_specifications.ConfigurationMetaNames.PACKAGE_EXTENSIONS: [{'guid': pkg_extn_id}],
   wml_client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {'guid': base_id}
}

sw_spec_details = wml_client.software_specifications.store(meta_props)
sw_spec_id = wml_client.software_specifications.get_id(sw_spec_details)

meta_data = {
    wml_client.repository.FunctionMetaNames.NAME: FUNCTION_NAME,
    wml_client.repository.FunctionMetaNames.DESCRIPTION: FUNCTION_NAME,
    wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sw_spec_id
}

function_details = wml_client.repository.store_function(meta_props=meta_data, function=msk_wd_pipeline)

function_id = wml_client.repository.get_function_uid(function_details)
function_id

base_id e4429883-c883-42b6-87a8-f419d64088cd


'fc65d340-186b-445e-a6e0-14030e0ae752'

### Deploy the function

In [15]:
meta_props_deployment = {
   wml_client.deployments.ConfigurationMetaNames.NAME: FUNCTION_DEPLOYMENT_NAME,
   wml_client.deployments.ConfigurationMetaNames.DESCRIPTION: FUNCTION_DEPLOYMENT_NAME,
   wml_client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {"name": "L", "num_nodes" : 20},

   wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

function_deployment_details = wml_client.deployments.create(artifact_uid=function_id, meta_props=meta_props_deployment)

function_deployment_id = wml_client.deployments.get_uid(function_deployment_details)
function_deployment_id
function_scoring_url = wml_client.deployments.get_details(function_deployment_id)['entity']['status']['online_url']['url']




#######################################################################################

Synchronous deployment creation for uid: 'fc65d340-186b-445e-a6e0-14030e0ae752' started

#######################################################################################


initializing....................................................................................................................................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='0d768527-f882-411b-b154-79fa481ad5f3'
------------------------------------------------------------------------------------------------




### Internal scoring URL of the deployed function
To get the external scoring url, in case you want to call this url from outside the environment replace <internal-nginx-svc:port> with your CPD envrionment URL

In [16]:
print(function_scoring_url)

https://internal-nginx-svc:12443/ml/v4/deployments/0d768527-f882-411b-b154-79fa481ad5f3/predictions


### Function to generate token needed to call the deployed function

In [17]:
import requests
from requests.auth import HTTPBasicAuth

def generate_access_token():
    headers={}
    headers["Accept"] = "application/json"
    
    #WARNING : Replace with your username and password
    auth = HTTPBasicAuth("user@mskcc.org", "password")

    #WARNING : Replace with your CPD environment URL
    ICP_TOKEN_URL= "https://https://cpd-cpd-cpd.apps.testcp4.mskcc.org/v1/preauth/validateAuth"
    
    response = requests.get(ICP_TOKEN_URL, headers=headers, auth=auth, verify=False)
    json_data = response.json()
    icp_access_token = json_data['accessToken']
    return icp_access_token

token = generate_access_token()

### Test if the deployed function is working properly

In [18]:
question_ = "What is the customer care number for HR?" 

In [19]:
%%time
import urllib3, requests, json

token = generate_access_token()
#print('Bearer ' + token)
# NOTE: you must construct mltoken based on provided documentation
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + token}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
#payload_scoring = {"question" :  'Will I accrue vacation time while I am on leave?'}
request_data = {"values": question_, "fields": "question"}

payload = {"input_data": [request_data]}

#response_scoring = requests.post(surl, json=payload, headers=header, verify=False)
response_scoring = requests.post(function_scoring_url+"?version=2021-08-21", json=payload, headers=header, verify=False)
print("Scoring response")
print(response_scoring.text)

Scoring response
{"trace": "4590706e85b2b015daf0e585ade592b9", "errors": [{"code": "invalid_authentication_credentials", "message": "The authentication credentials provided are invalid. Retry with correct credentials.", "target": {"type": "header", "name": "Authorization"}}], "status_code": 400}

CPU times: user 10.4 ms, sys: 1.88 ms, total: 12.3 ms
Wall time: 580 ms


[Back to top](#top)